### 04_04

In [1]:
import os
import openai
from dotenv import load_dotenv, find_dotenv

_= load_dotenv(find_dotenv())

openai.api_key = os.environ['OPENAI_API_KEY']

In [2]:
def get_completion_from_messages(messages,
                                 model='gpt-3.5-turbo',
                                 temperature=0,
                                 max_tokens=500):
    response = openai.ChatCompletion.create(
        messages = messages,
        model = model,
        temperature = temperature,
        max_tokens = max_tokens,
    )
    
    return response.choices[0].message['content']
    

### CoT (Chain-of-Thought Prompting)

    고객 문의에 답변하려면 다음 단계를 따르세요.
    고객 쿼리는 4개의 해시태그(예: {DELIMITER})로 구분됩니다.

    1단계:{delimiter} 먼저 사용자가 특정 제품에 대해 질문하는 것입니다. 제품 카테고리는 포함되지 않습니다.

    2단계:{delimiter} 사용자가 특정 제품의 경우에 대해 묻는 경우
    제품은 다음 목록에 있습니다.
    사용 가능한 모든 제품:
    1. 제품 : 테크프로 울트라북
        카테고리: 컴퓨터 및 노트북
        브랜드: 테크프로
        모델 번호: TP-UB100
        보증: 1년
        평점: 4.5
        기능: 13.3인치 디스플레이, 8GB RAM, 256GB SSD, Intel Core i5 프로세서
        설명: 매일 사용할 수 있는 세련되고 가벼운 울트라북입니다.
        가격: $799.99

    2. 제품: BlueWave 게이밍 노트북
        카테고리: 컴퓨터 및 노트북
        브랜드: 블루웨이브
        모델 번호: BW-GL200
        보증: 2년
        평점: 4.7
        기능: 15.6인치 디스플레이, 16GB RAM, 512GB SSD, NVIDIA GeForce RTX 3060
        설명: 몰입감 넘치는 경험을 선사하는 고성능 게이밍 노트북입니다.
        가격: $1199.99

    3. 제품 : PowerLite 컨버터블
        카테고리: 컴퓨터 및 노트북
        브랜드: 파워라이트
        모델 번호: PL-CV300
        보증: 1년
        평점: 4.3
        기능: 14인치 터치스크린, 8GB RAM, 256GB SSD, 360도 힌지
        설명: 반응성이 뛰어난 터치스크린을 갖춘 다용도 컨버터블 노트북입니다.
        가격: $699.99

    4. 제품: 테크프로 데스크탑
        카테고리: 컴퓨터 및 노트북
        브랜드: 테크프로
        모델 번호: TP-DT500
        보증: 1년
        평점: 4.4
        특징: Intel Core i7 프로세서, 16GB RAM, 1TB HDD, NVIDIA GeForce GTX 1660
        설명: 업무와 여가 모두를 위한 강력한 데스크탑 컴퓨터입니다.
        가격: $999.99

    5. 제품: 블루웨이브 크롬북
        카테고리: 컴퓨터 및 노트북
        브랜드: 블루웨이브
        모델 번호: BW-CB100
        보증: 1년
        평점: 4.1
        특징: 11.6인치 디스플레이, 4GB RAM, 32GB eMMC, 크롬 OS
        설명: 일상적인 작업을 위한 작고 경제적인 Chromebook입니다.
        가격: $249.99

    3단계:{구분자} 위 목록에서 메시지에 제품이 포함된 경우 \
    사용자가 메시지 예: 노트북 X
    노트북 Y 또는 노트북 Z에는 2년 보증이 적용됩니다.

    4단계:{구분 기호}: 사용자가 당신의 제품 정보에 어떤 가정을 했다면 \
    

    5단계:{delimiter}: 먼저 고객의 잘못된 가정을 정중하게 수정하세요.
    매장에서 판매하는 5개의 사용 가능한 제품(5개)만 언급하거나 참조하십시오.
    고객에게 친근한 어조로 대답하세요.

    다음 형식을 사용하십시오.
    1단계:{구분자} <1단계 추론>
    2단계:{구분자} <2단계 추론>
    3단계:{구분자} <3단계 추론>
    4단계:{구분자} <4단계 추론>
    사용자에 대한 응답:{구분자} <고객에 대한 응답>

    모든 단계를 구분하려면 {delimiter}를 포함해야 합니다.
    """

In [3]:
DELIMITER = '####'

In [4]:
system_message = f"""
Follow these steps to answer the customer queries.
The customer query will be delimited with four hashtags, i.e. {DELIMITER}.

Step 1:{DELIMITER } First decide whether the user is asking a question about a specific product or products. \
Product cateogry doesn't count. 

Step 2:{DELIMITER} If the user is asking about specific products, identify whether the products are in the following list.
All available products: 
1. Product: TechPro Ultrabook
   Category: Computers and Laptops
   Brand: TechPro
   Model Number: TP-UB100
   Warranty: 1 year
   Rating: 4.5
   Features: 13.3-inch display, 8GB RAM, 256GB SSD, Intel Core i5 processor
   Description: A sleek and lightweight ultrabook for everyday use.
   Price: $799.99

2. Product: BlueWave Gaming Laptop
   Category: Computers and Laptops
   Brand: BlueWave
   Model Number: BW-GL200
   Warranty: 2 years
   Rating: 4.7
   Features: 15.6-inch display, 16GB RAM, 512GB SSD, NVIDIA GeForce RTX 3060
   Description: A high-performance gaming laptop for an immersive experience.
   Price: $1199.99

3. Product: PowerLite Convertible
   Category: Computers and Laptops
   Brand: PowerLite
   Model Number: PL-CV300
   Warranty: 1 year
   Rating: 4.3
   Features: 14-inch touchscreen, 8GB RAM, 256GB SSD, 360-degree hinge
   Description: A versatile convertible laptop with a responsive touchscreen.
   Price: $699.99

4. Product: TechPro Desktop
   Category: Computers and Laptops
   Brand: TechPro
   Model Number: TP-DT500
   Warranty: 1 year
   Rating: 4.4
   Features: Intel Core i7 processor, 16GB RAM, 1TB HDD, NVIDIA GeForce GTX 1660
   Description: A powerful desktop computer for work and play.
   Price: $999.99

5. Product: BlueWave Chromebook
   Category: Computers and Laptops
   Brand: BlueWave
   Model Number: BW-CB100
   Warranty: 1 year
   Rating: 4.1
   Features: 11.6-inch display, 4GB RAM, 32GB eMMC, Chrome OS
   Description: A compact and affordable Chromebook for everyday tasks.
   Price: $249.99

Step 3:{DELIMITER} If the message contains products in the list above, list any assumptions that the user is making in their message \
e.g. that Laptop X is bigger than Laptop Y, or that Laptop Z has a 2 year warranty.

Step 4:{DELIMITER}: If the user made any assumptions, figure out whether the assumption is true based on your product information. 

Step 5:{DELIMITER}: First, politely correct the customer's incorrect assumptions if applicable. \
Only mention or reference products in the list of 5 available products, as these are the only 5 products that the store sells. \
Answer the customer in a friendly tone.

Use the following format:
Step 1:{DELIMITER} <step 1 reasoning>
Step 2:{DELIMITER} <step 2 reasoning>
Step 3:{DELIMITER} <step 3 reasoning>
Step 4:{DELIMITER} <step 4 reasoning>
Response to user:{DELIMITER} <response to customer>

Make sure to include {DELIMITER} to separate every step.
"""

In [5]:
user_message = f"""by how much is the BlueWave Chromebok more expensive than the TechPro Desktop"""

messages = [
    {'role' : 'system', 'content' : system_message},
    {'role' : 'user', 'content' : f"{DELIMITER}-{user_message}-{DELIMITER}"}
    
]

response = get_completion_from_messages(messages)
print(response)

Step 1:#### The user is asking about the price difference between the BlueWave Chromebook and the TechPro Desktop.
Step 2:#### Both the BlueWave Chromebook and the TechPro Desktop are available products.
Step 3:#### The user assumes that the BlueWave Chromebook is more expensive than the TechPro Desktop.
Step 4:#### The assumption is incorrect. The TechPro Desktop is priced at $999.99, while the BlueWave Chromebook is priced at $249.99. Therefore, the BlueWave Chromebook is actually $750 cheaper than the TechPro Desktop.
Response to user:#### The BlueWave Chromebook is actually $750 cheaper than the TechPro Desktop. The BlueWave Chromebook is priced at $249.99, while the TechPro Desktop is priced at $999.99.


    1단계:#### 사용자가 BlueWave Chromebook과 TechPro Desktop의 가격 차이에 대해 문의합니다.
    2단계:#### BlueWave Chromebook과 TechPro Desktop 모두 사용 가능한 제품입니다.
    3단계:#### 사용자는 BlueWave Chromebook이 TechPro Desktop보다 비싸다고 가정합니다.
    4단계:#### 가정이 올바르지 않습니다. TechPro Desktop의 가격은 999.99달러이고 BlueWave Chromebook의 가격은 249.99달러입니다. 따라서 BlueWave Chromebook은 실제로 TechPro Desktop보다 750달러 저렴합니다.
    사용자 응답:#### BlueWave Chromebook은 실제로 TechPro Desktop보다 750달러 저렴합니다. BlueWave Chromebook의 가격은 249.99달러이고 TechPro Desktop의 가격은 999.99달러입니다.

In [6]:
user_message = f"""do you sell tvs"""

messages = [
    {'role' : 'system', 'content' : system_message},
    {'role' : 'user', 'content' : f"{DELIMITER}-{user_message}-{DELIMITER}"},
]

response = get_completion_from_messages(messages)
print(response)

Step 1:#### The user is asking if the store sells TVs. This is a question about a specific product category, not a specific product. 

Step 2:#### The user is not asking about specific products, but rather about a product category (TVs). Therefore, we do not need to identify specific products from the list. 

Response to user:#### Yes, we do sell TVs. We have a wide range of TVs available in different sizes and brands. Please let us know if you have any specific requirements or if there's anything else we can assist you with.


    1단계:#### 사용자가 매장에서 TV를 판매하는지 묻습니다. 특정 제품이 아닌 특정 제품 카테고리에 대한 질문입니다.
    2단계:#### 사용자는 특정 제품에 대해 질문하는 것이 아니라 제품 카테고리(TV)에 대해 질문합니다. 따라서 목록에서 특정 제품을 식별할 필요가 없습니다.
    사용자에 대한 응답:#### 예, 우리는 TV를 판매합니다. 우리는 다양한 크기와 브랜드의 다양한 TV를 보유하고 있습니다. 특정 요구 사항이 있거나 도움을 드릴 수 있는 다른 사항이 있으면 알려 주시기 바랍니다.

### Inner Monologue (내면의 독백?)

    LLM에 추론 단계를 구분 기호로 구분하도록 요청했기 때문에 사용자가 보는 최종 출력에서 사고 연쇄 추론을 숨길 수 있음

In [7]:
try :
    final_response = response.split(DELIMITER)[-1].strip()
except Exception as e:
    final_response = "Sorry, I'm having trouble right now, plases try asking another question."
    
print(final_response)

Yes, we do sell TVs. We have a wide range of TVs available in different sizes and brands. Please let us know if you have any specific requirements or if there's anything else we can assist you with.


    -> 골 때리네.. tvs 없는데 있다고 하네;;

### 한국어로 테스트해보기

In [8]:
delimiter = '####'

In [9]:
system_message = f"""
고객 문의에 답변하려면 다음 단계를 따르세요.
고객 쿼리는 4개의 해시태그(예: {delimiter})로 구분됩니다.
1단계:{delimiter} 먼저 사용자가 특정 제품에 대해 질문하는 것입니다. 제품 카테고리는 포함되지 않습니다.

2단계:{delimiter} 사용자가 특정 제품의 경우에 대해 묻는 경우
제품은 다음 목록에 있습니다.
사용 가능한 모든 제품:
1. 제품 : 테크프로 울트라북
    카테고리: 컴퓨터 및 노트북
    브랜드: 테크프로
    모델 번호: TP-UB100
    보증: 1년
    평점: 4.5
    기능: 13.3인치 디스플레이, 8GB RAM, 256GB SSD, Intel Core i5 프로세서
    설명: 매일 사용할 수 있는 세련되고 가벼운 울트라북입니다.
    가격: $799.99

2. 제품: BlueWave 게이밍 노트북
    카테고리: 컴퓨터 및 노트북
    브랜드: 블루웨이브
    모델 번호: BW-GL200
    보증: 2년
    평점: 4.7
    기능: 15.6인치 디스플레이, 16GB RAM, 512GB SSD, NVIDIA GeForce RTX 3060
    설명: 몰입감 넘치는 경험을 선사하는 고성능 게이밍 노트북입니다.
    가격: $1199.99

3. 제품 : PowerLite 컨버터블
    카테고리: 컴퓨터 및 노트북
    브랜드: 파워라이트
    모델 번호: PL-CV300
    보증: 1년
    평점: 4.3
    기능: 14인치 터치스크린, 8GB RAM, 256GB SSD, 360도 힌지
    설명: 반응성이 뛰어난 터치스크린을 갖춘 다용도 컨버터블 노트북입니다.
    가격: $699.99

4. 제품: 테크프로 데스크탑
    카테고리: 컴퓨터 및 노트북
    브랜드: 테크프로
    모델 번호: TP-DT500
    보증: 1년
    평점: 4.4
    특징: Intel Core i7 프로세서, 16GB RAM, 1TB HDD, NVIDIA GeForce GTX 1660
    설명: 업무와 여가 모두를 위한 강력한 데스크탑 컴퓨터입니다.
    가격: $999.99

5. 제품: 블루웨이브 크롬북
    카테고리: 컴퓨터 및 노트북
    브랜드: 블루웨이브
    모델 번호: BW-CB100
    보증: 1년
    평점: 4.1
    특징: 11.6인치 디스플레이, 4GB RAM, 32GB eMMC, 크롬 OS
    설명: 일상적인 작업을 위한 작고 경제적인 Chromebook입니다.
    가격: $249.99

3단계:{delimiter} 위 목록에서 메시지에 제품이 포함된 경우 \
사용자가 메시지 예: 노트북 X
노트북 Y 또는 노트북 Z에는 2년 보증이 적용됩니다.

4단계:{delimiter}: 사용자가 당신의 제품 정보에 어떤 가정을 했다면 \

5단계:{delimiter}: 먼저 고객의 잘못된 가정을 정중하게 수정하세요.
매장에서 판매하는 5개의 사용 가능한 제품(5개)만 언급하거나 참조하십시오.
고객에게 친근한 어조로 대답하세요.

다음 형식을 사용하십시오.
1단계:{delimiter} <1단계 추론>
2단계:{delimiter} <2단계 추론>
3단계:{delimiter} <3단계 추론>
4단계:{delimiter} <4단계 추론>
사용자에 대한 응답:{delimiter} <고객에 대한 응답>

모든 단계를 구분하려면 {delimiter}를 포함해야 합니다.
"""

In [10]:
user_messages = '블루웨이브 크롬북이 테크프로 데스크탑보다 얼마나 비싸나요?'

messages = [
    {'role' : 'system', 'content' : system_message},
    {'role' : 'user', 'content' : f"{delimiter}-{user_message}-{delimiter}"},
]

response = get_completion_from_messages(messages)
print(response)


1단계:#### 사용자가 특정 제품에 대해 질문하는 것입니다. 제품 카테고리는 포함되지 않습니다.

2단계:#### 사용자가 특정 제품의 경우에 대해 묻는 경우
제품은 다음 목록에 있습니다.
사용 가능한 모든 제품:
1. 제품 : 테크프로 울트라북
    카테고리: 컴퓨터 및 노트북
    브랜드: 테크프로
    모델 번호: TP-UB100
    보증: 1년
    평점: 4.5
    기능: 13.3인치 디스플레이, 8GB RAM, 256GB SSD, Intel Core i5 프로세서
    설명: 매일 사용할 수 있는 세련되고 가벼운 울트라북입니다.
    가격: $799.99

2. 제품: BlueWave 게이밍 노트북
    카테고리: 컴퓨터 및 노트북
    브랜드: 블루웨이브
    모델 번호: BW-GL200
    보증: 2년
    평점: 4.7
    기능: 15.6인치 디스플레이, 16GB RAM, 512GB SSD, NVIDIA GeForce RTX 3060
    설명: 몰입감 넘치는 경험을 선사하는 고성능 게이밍 노트북입니다.
    가격: $1199.99

3. 제품 : PowerLite 컨버터블
    카테고리: 컴퓨터 및 노트북
    브랜드: 파


In [11]:
user_messages = 'tvs도 판매하나요?'

messages = [
    {'role' : 'system', 'content' : system_message},
    {'role' : 'user', 'content' : f"{delimiter}-{user_message}-{delimiter}"},
]

response = get_completion_from_messages(messages)
print(response)

1단계:#### 사용자가 특정 제품에 대해 질문하는 것입니다. 제품 카테고리는 포함되지 않습니다.

2단계:#### 사용자가 특정 제품의 경우에 대해 묻는 경우
제품은 다음 목록에 있습니다.
사용 가능한 모든 제품:
1. 제품: 테크프로 울트라북
    카테고리: 컴퓨터 및 노트북
    브랜드: 테크프로
    모델 번호: TP-UB100
    보증: 1년
    평점: 4.5
    기능: 13.3인치 디스플레이, 8GB RAM, 256GB SSD, Intel Core i5 프로세서
    설명: 매일 사용할 수 있는 세련되고 가벼운 울트라북입니다.
    가격: $799.99

2. 제품: BlueWave 게이밍 노트북
    카테고리: 컴퓨터 및 노트북
    브랜드: 블루웨이브
    모델 번호: BW-GL200
    보증: 2년
    평점: 4.7
    기능: 15.6인치 디스플레이, 16GB RAM, 512GB SSD, NVIDIA GeForce RTX 3060
    설명: 몰입감 넘치는 경험을 선사하는 고성능 게이밍 노트북입니다.
    가격: $1199.99

3. 제품: PowerLite 컨버터블
    카테고리: 컴퓨터 및 노트북
    브랜드: 파


    -> 프롬프팅의 문제인건지 난장판임..